In [1]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path
from datetime import datetime

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary
from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16899571712
0
0


In [2]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

# root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"
root_dir = parent_path + "occ_sim_dataset/"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [4]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
#         A.Resize(640, 480)  # Resize all images to be 640x480
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [5]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(0.95, 0.025, 0.025), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  

In [6]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_file = self.imgs_files[idx]
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original, img_file
        else:
            return img, target, img_file
    
    def __len__(self):
        return len(self.imgs_files)
    

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import math
import numpy as np
from torch.autograd import Variable
import torch_geometric.nn as pyg
from torch_geometric.data import Data

_EPS = 1e-10

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(True),
            nn.Linear(64, hidden_dim)
        )

    def forward(self, x):
        # x: (batch_size, seq_len, hidden_dim)
        energy = self.projection(x)  # (batch_size, seq_len, 1)
        weights = func.softmax(energy, dim=-1)  # (batch_size, seq_len)
        outputs = (x * weights)  # (batch_size, hidden_dim)
        return outputs, weights

class AttentionMLP(nn.Module):
    def __init__(self, n_in, n_hid, n_out, do_prob=0.):
        super(AttentionMLP, self).__init__()
        self.fc1 = nn.Linear(n_in, n_hid)
        self.attn = SelfAttention(n_hid)
        self.fc2 = nn.Linear(n_hid, n_out)
        self.bn = nn.BatchNorm1d(n_out)
        self.dropout_prob = do_prob
        
    def batch_norm(self, inputs):
        x = inputs.view(inputs.size(0) * inputs.size(1), -1)
        x = self.bn(x)
        return x.view(inputs.size(0), inputs.size(1), -1)

    def forward(self, inputs):
        x = func.elu(self.fc1(inputs))
        print("shape of x before attention", x.shape)
        x, _ = self.attn(x)
        print("shape of x after attn", x.shape)
        x = func.dropout(x, self.dropout_prob, training=self.training)
        x = func.elu(self.fc2(x))
        print("batch norm input", x.shape)
        return self.batch_norm(x)

class GraphEncoder(nn.Module):
    def __init__(self, n_in, n_hid, n_out=4, do_prob=0., factor=True):
        super(GraphEncoder, self).__init__()

        self.factor = factor

        self.mlp1 = AttentionMLP(n_in, n_hid, n_hid, do_prob)
        self.mlp2 = AttentionMLP(n_hid * 2, n_hid, n_hid, do_prob)
        self.mlp3 = AttentionMLP(n_hid, n_hid, n_hid, do_prob)
        if self.factor:
            self.mlp4 = AttentionMLP(n_hid * 3, n_hid, n_hid, do_prob)
            print("Using factor graph MLP encoder.")
        else:
            self.mlp4 = AttentionMLP(n_hid * 2, n_hid, n_hid, do_prob)
#             print("mlp4", self.mlp4)
            print("Using MLP graph encoder.")
        self.fc_out = nn.Linear(n_hid, n_out)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.fill_(0.1)

    def edge2node(self, x, rel_rec, rel_send):
        # NOTE: Assumes that we have the same graph across all samples.
        incoming = torch.matmul(rel_rec.t(), x)
#         print("Nodes in edge2node: ", incoming)
#         print("final incoming: ", incoming / incoming.size(1))
        return incoming / incoming.size(1)

    def node2edge(self, x, rel_rec, rel_send):
        # NOTE: Assumes that we have the same graph across all samples.
        receivers = torch.matmul(rel_rec, x)
        senders = torch.matmul(rel_send, x)
        edges = torch.cat([receivers, senders], dim=2)
#         print("Edges in node2edge: ", edges)
        return edges

    def forward(self, inputs, rel_rec, rel_send):
        # Input shape: [num_sims, num_atoms, num_timesteps, num_dims]
        x = inputs.view(inputs.size(0), inputs.size(1), -1)
#         print("x shape:", x.shape)
#         print("rel_rec shape:", rel_rec.shape)
#         print("rel_send shape:", rel_send.shape)

        # New shape: [num_sims, num_atoms, num_timesteps*num_dims]
        x = self.mlp1(x)  # 2-layer ELU net per node

        x = self.node2edge(x, rel_rec, rel_send)
        x = self.mlp2(x)
        x_skip = x    
        
        if self.factor:
            x = self.edge2node(x, rel_rec, rel_send)
            x = self.mlp3(x)
            x = self.node2edge(x, rel_rec, rel_send)
            x = torch.cat((x, x_skip), dim=2)  # Skip connection
            x = self.mlp4(x)
        else:
            x = self.mlp3(x)
            x = torch.cat((x, x_skip), dim=2)  # Skip connection
            x = self.mlp4(x)
            
#         print("output of graph encoder: ", self.fc_out(x))

        return self.fc_out(x)    
    
class GraphDecoder(nn.Module):

    def __init__(self, n_in_node, edge_types, msg_hid, msg_out, n_hid,
                 do_prob=0., skip_first=False):
        super(GraphDecoder, self).__init__()
        self.msg_fc1 = nn.ModuleList(
            [nn.Linear(2 * n_in_node, msg_hid) for _ in range(edge_types)])
        self.msg_fc2 = nn.ModuleList(
            [nn.Linear(msg_hid, msg_out) for _ in range(edge_types)])
        self.msg_out_shape = msg_out
        self.skip_first_edge_type = skip_first

        self.out_fc1 = nn.Linear(n_in_node + msg_out, n_hid)
        self.out_fc2 = nn.Linear(n_hid, n_hid)
        self.out_fc3 = nn.Linear(n_hid, n_in_node)

        print('Using learned graph decoder.')

        self.dropout_prob = do_prob

    def single_step_forward(self, single_timestep_inputs, rel_rec, rel_send,
                            single_timestep_rel_type):

        # single_timestep_inputs has shape
        # [batch_size, num_timesteps, num_atoms, num_dims]

        # single_timestep_rel_type has shape:
        # [batch_size, num_timesteps, num_atoms*(num_atoms-1), num_edge_types]

        # Node2edge
        receivers = torch.matmul(rel_rec, single_timestep_inputs)
        senders = torch.matmul(rel_send, single_timestep_inputs)
        pre_msg = torch.cat([receivers, senders], dim=-1)

        all_msgs = Variable(torch.zeros(pre_msg.size(0), pre_msg.size(1),self.msg_out_shape))
        if single_timestep_inputs.is_cuda:
            all_msgs = all_msgs.cuda()

        if self.skip_first_edge_type:
            start_idx = 1
        else:
            start_idx = 0

        # Run separate MLP for every edge type
        # NOTE: To exlude one edge type, simply offset range by 1
        for i in range(start_idx, len(self.msg_fc2)):
            msg = func.relu(self.msg_fc1[i](pre_msg))
            msg = func.dropout(msg, p=self.dropout_prob)
            msg = func.relu(self.msg_fc2[i](msg))
            msg = msg * single_timestep_rel_type[:, :, i:i + 1]
            all_msgs += msg

        # Aggregate all msgs to receiver
        agg_msgs = all_msgs.transpose(-2, -1).matmul(rel_rec).transpose(-2, -1)
        agg_msgs = agg_msgs.contiguous()

        # Skip connection
        aug_inputs = torch.cat([single_timestep_inputs, agg_msgs], dim=-1)

        # Output MLP
        pred = func.dropout(func.relu(self.out_fc1(aug_inputs)), p=self.dropout_prob)
        pred = func.dropout(func.relu(self.out_fc2(pred)), p=self.dropout_prob)
        pred = self.out_fc3(pred)
#        print(pred.shape,single_timestep_inputs.shape)
#         print("output for single time steps fwd: ", single_timestep_inputs, pred, single_timestep_inputs + pred)
        # Predict position/velocity difference
        return single_timestep_inputs + pred

    def forward(self, inputs, rel_type, rel_rec, rel_send, pred_steps=4):
        # NOTE: Assumes that we have the same graph across all samples.


        # Only take n-th timesteps as starting points (n: pred_steps)
        last_pred = inputs[:, :, :]
        #asa
        curr_rel_type = rel_type[:, :, :]
        preds=[]
        #print(curr_rel_type.shape)
        # NOTE: Assumes rel_type is constant (i.e. same across all time steps).

        # Run n prediction steps
        for step in range(0, pred_steps):
            last_pred = self.single_step_forward(last_pred, rel_rec, rel_send,
                                                 curr_rel_type)
            preds.append(last_pred)

        sizes = [preds[0].size(0), preds[0].size(1),
                 preds[0].size(2)]

        output = Variable(torch.zeros(sizes))
        if inputs.is_cuda:
            output = output.cuda()

        # Re-assemble correct timeline
        for i in range(len(preds)):
            output[:, :, :] = preds[i]

        pred_all = output[:, :, :]

        # NOTE: We potentially over-predicted (stored in future_pred). Unused.
        # future_pred = output[:, (inputs.size(1) - 1):, :, :]
#         print("output for pred_all", pred_all)
        return pred_all#.transpose(1, 2).contiguous()    


In [8]:
def my_softmax(input, axis=1):
    trans_input = input.transpose(axis, 0).contiguous()
    soft_max_1d = func.softmax(trans_input,dim=0)
    return soft_max_1d.transpose(axis, 0)


In [9]:
class KeypointPipeline(nn.Module):
    def __init__(self, weights_path):
        super(KeypointPipeline, self).__init__()  
        self.keypoint_model = torch.load(weights_path).to(device)
        self.encoder = GraphEncoder(4,512,4,0.5,True)
        self.decoder = GraphDecoder(n_in_node=4,
                                 edge_types=2,
                                 msg_hid=512,
                                 msg_out=512,
                                 n_hid=512,
                                 do_prob=0.5,
                                 skip_first=False)
        
#         self.off_diag = np.ones([6,6]) - np.eye(6)

#         self.rel_rec = np.array(encode_onehot(np.where(self.off_diag)[1]), dtype=np.float32)
#         self.rel_send = np.array(encode_onehot(np.where(self.off_diag)[0]), dtype=np.float32)
#         self.rel_rec = torch.FloatTensor(self.rel_rec)
#         self.rel_send = torch.FloatTensor(self.rel_send)

        num_nodes = 6
        self.off_diag = np.zeros([num_nodes, num_nodes])        
#         # Creating a cycle: 1->2, 2->3, ..., 6->1
#         for i in range(num_nodes):
#             self.off_diag[i, (i + 1) % num_nodes] = 1

        # Creating a bidirectional cycle
#         for i in range(num_nodes):
#             # Forward connection: i -> (i + 1) % num_nodes
#             self.off_diag[i, (i + 1) % num_nodes] = 1

#             # Backward connection: i -> (i - 1 + num_nodes) % num_nodes
#             # The addition of num_nodes before modulo ensures a positive index
#             self.off_diag[i, (i - 1 + num_nodes) % num_nodes] = 1

        # Creating a bidirectional, non-cyclic graph
        for i in range(num_nodes):
            # Forward connection: i -> (i + 1), except for the last node
            if i < num_nodes - 1:  # This prevents connecting the last node to the first
                self.off_diag[i, i + 1] = 1

            # Backward connection: i -> (i - 1), except for the first node
            if i > 0:  # This prevents connecting the first node to the last
                self.off_diag[i, i - 1] = 1

        # Update rel_rec and rel_send based on the new off_diag
        self.rel_rec = np.array(encode_onehot(np.where(self.off_diag)[1]), dtype=np.float32)
        self.rel_send = np.array(encode_onehot(np.where(self.off_diag)[0]), dtype=np.float32)
        self.rel_rec = torch.FloatTensor(self.rel_rec).to(device)
        self.rel_send = torch.FloatTensor(self.rel_send).to(device)

        self.encoder= self.encoder.cuda()
        self.decoder = self.decoder.cuda()
        self.rel_rec = self.rel_rec.cuda()
        self.rel_send = self.rel_send.cuda()
    
    def process_model_output(self, output):
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist()

        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], 
                                            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy()

        confidence = output[0]['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        labels = output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        keypoints = []
        for idx, kps in enumerate(output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
            keypoints.append(list(map(int, kps[0,0:2])) + [confidence[idx]] + [labels[idx]])
        
        # Sort keypoints based on label
        keypoints.sort(key=lambda x: x[-1])
        print("Keypoints: ", keypoints)
        return keypoints
    
    def keypoints_to_graph(self, keypoints, image_width, image_height):
        # keypoints is expected to be a tensor with shape (num_keypoints, 4),
        # where each keypoint is (x, y, score, label).
        # Convert all elements in keypoints to tensors if they are not already
        keypoints = [torch.tensor(kp, dtype=torch.float32).to(device) if not isinstance(kp, torch.Tensor) else kp for kp in keypoints]

        # Then stack them
        keypoints = torch.stack(keypoints).to(device)        
        
        # Remove duplicates: Only keep the keypoint with the highest score for each label
        unique_labels, best_keypoint_indices = torch.unique(keypoints[:, 3], return_inverse=True)
        best_scores, best_indices = torch.max(keypoints[:, 2].unsqueeze(0) * (best_keypoint_indices == torch.arange(len(unique_labels)).unsqueeze(1).cuda()), dim=1)
        keypoints = keypoints[best_indices]
        
#         print("init keypoints in graph features", keypoints)

        # Normalize x and y to be in the range [-1, 1]
        keypoints[:, 0] = (keypoints[:, 0] - image_width / 2) / (image_width / 2)
        keypoints[:, 1] = (keypoints[:, 1] - image_height / 2) / (image_height / 2)

        # Use only x, y, and score for the graph features
        graph_features = keypoints[:, :4]  # Now shape is (num_keypoints, 3)
        
        # Ensure the shape is [num_keypoints, 3] before returning
        graph_features = graph_features.view(-1, 4)  # Reshape to ensure it's [num_keypoints, 3]
#         print("graph features", graph_features)
#         print("graph features shape", graph_features.shape)
        print("graph feature: ", graph_features)

        return graph_features
        
    def forward(self, imgs):
        # Temporarily set the keypoint model to evaluation mode
        keypoint_model_training = self.keypoint_model.training
        self.keypoint_model.eval()

        # Process each image in the batch
        with torch.no_grad():
            batch_outputs = [self.keypoint_model(img.unsqueeze(0).to(device)) for img in imgs]

        # Set the keypoint model back to its original training mode
        self.keypoint_model.train(mode=keypoint_model_training)

        # Process model outputs to get labeled keypoints
        batch_labeled_keypoints = [self.process_model_output(output) for output in batch_outputs]
        # Generate graph input tensor for each image and handle varying number of keypoints
        batch_x = []
        for labeled_keypoints in batch_labeled_keypoints:
            keypoints = self.keypoints_to_graph(labeled_keypoints, 640, 480)

            # Initialize x with zeros for 6 nodes with 4 features each
            x = torch.zeros(1, 6, 4, device=device)

            # Ensure that keypoints are on the correct device and fill in x
            num_keypoints_detected = keypoints.size(0)
            if num_keypoints_detected <= 6:
                x[0, :num_keypoints_detected, :] = keypoints
            else:
                raise ValueError("Number of keypoints detected exceeds the maximum of 6.")

            batch_x.append(x)

        # Stack the batch of x tensors for batch processing
        batch_x = torch.cat(batch_x, dim=0)

        # Forward pass through the encoder and decoder
        logits = self.encoder(batch_x, self.rel_rec, self.rel_send)
        edges = my_softmax(logits, -1)
        KGNN2D = self.decoder(batch_x, edges, self.rel_rec, self.rel_send)

        return logits, KGNN2D, batch_labeled_keypoints

In [10]:
def loss_edges(valid_points, edges):
#     num_nodes = 6
    print("valid_points: ", valid_points)
    batch_size, num_nodes = valid_points.shape
    print(batch_size, num_nodes)
    off_diag = np.zeros([num_nodes, num_nodes])    
    print("valid_points shape: ", valid_points.shape)
    print("off_diag shape: ", off_diag.shape)

    # Creating a bidirectional, cyclic graph
#     for i in range(num_nodes):
#         next_node = (i + 1) % num_nodes  # Ensures cyclic behavior
#         off_diag[i, next_node] = 1  # Connect node i to node i+1 (or to the first node if i is the last node)
#         off_diag[next_node, i] = 1  # Connect node i+1 (or the first node if i is the last node) back to node i

#     Creating a bidirectional, non-cyclic graph
    for i in range(num_nodes):
        # Forward connection: i -> (i + 1), except for the last node
        if i < num_nodes - 1:  # This prevents connecting the last node to the first
            off_diag[i, i + 1] = 1

        # Backward connection: i -> (i - 1), except for the first node
        if i > 0:  # This prevents connecting the first node to the last
            off_diag[i, i - 1] = 1   
        
#     print("off_diag original: ", off_diag)
        
    # Convert off_diag to tensor for indexing
    off_diag = torch.tensor(off_diag, dtype=torch.bool, device='cuda')
    
#     print("off_diag: ", off_diag.shape, off_diag)
    # Extracting indices where there is a relationship
    idx = torch.where(off_diag)[1].reshape((num_nodes-1), -1).to(device='cuda', dtype=torch.long)
#     print(torch.where(off_diag)[1])
#     print("idx: ", idx)# Adjust relations calculation to accommodate batches and node relationships
    relations = torch.zeros((batch_size, (num_nodes-1) * 2), device='cuda')
#     print("relations: ", relations)
    
    for count, vis in enumerate(valid_points):
        vis = vis.view(-1, 1).float()
#         print("vis: ", vis)
        vis_tran_mat = vis*vis.t() # Matrix multiplication to get visibility matrix
#         print("vis after vis*vis.t(): ", vis_tran_mat)
        vis_selected = torch.gather(vis_tran_mat, 1, idx)  # Gather visible relations based on the bidirectional cyclic graph
#         print("vis after gather: ", vis_selected)
#         print("vis with flatten: ", vis_selected.view(-1))
        relations[count] = vis_selected.view(-1)  # Flatten and assign
#     print("relations", relations.shape)
    relations = relations.to(torch.long)  # Ensure correct dtype for loss calculation
    # Calculate and return cross-entropy loss
#     print("edges shape: ", edges.shape, edges)
    print("relations reshaped", relations.view(-1).shape)
    print("edges reshaped", edges.view(-1,4).shape)
    loss_edges = func.cross_entropy(edges.view(-1, 4), relations.view(-1))
    return loss_edges



In [11]:
# def nll_gaussian(preds, target, variance, add_const=False):
#     neg_log_p = ((preds - target) ** 2 / (2 * variance))
#     if add_const:
#         const = 0.5 * np.log(2 * np.pi * variance)
#         neg_log_p += const
#     return neg_log_p.sum() / (target.size(0) * target.size(1))

# def kgnn2d_loss(keypoints_gt, valid_points, keypoints_logits):
#     # Ensure data types are consistent and move tensors to the appropriate device
#     keypoints_gt = keypoints_gt.type(torch.FloatTensor).cuda()
#     keypoints_logits = keypoints_logits.type(torch.FloatTensor).cuda()
#     valid_points = valid_points.type(torch.FloatTensor).cuda()

#     # Print shapes for debugging
# #     print(f"keypoints_gt.shape: {keypoints_gt.shape}")
# #     print(f"keypoints_logits.shape: {keypoints_logits.shape}")
# #     print(f"valid_points.shape: {valid_points.shape}")
#     keypoints_gt = keypoints_gt.type(torch.FloatTensor)*valid_points.unsqueeze(2).type(torch.FloatTensor)
#     keypoints_logits = keypoints_logits.type(torch.FloatTensor)*valid_points.unsqueeze(2).type(torch.FloatTensor)
#     keypoints_gt = keypoints_gt.cuda()
#     keypoints_logits = keypoints_logits.cuda()
#     loss_occ = nll_gaussian(keypoints_gt[:,:,0:2], keypoints_logits[:,:,0:2] , 0.1)
#     return loss_occ

In [12]:
def kgnn2d_loss(gt_keypoints, pred_keypoints):
    loss = func.mse_loss(pred_keypoints, gt_keypoints)

    return loss

In [13]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot
# def process_keypoints(keypoints):
#     # Assuming keypoints is a list of Nx3 tensors where N is the number of keypoints
#     # and each keypoint is represented as [x, y, visibility]
#     # Remove the unnecessary middle dimension
#     keypoints = [kp.squeeze(1) for kp in keypoints]
#     visibilities = [kp[:, 2] for kp in keypoints]  # Extract visibility flags
#     valid_vis_all = torch.cat([v == 1 for v in visibilities]).long().cuda()
#     valid_invis_all = torch.cat([v == 0 for v in visibilities]).long().cuda()

#     keypoints_gt = torch.cat([kp[:, :2] for kp in keypoints]).float().cuda()  # Gather all keypoints and discard visibility flags
#     keypoints_gt = keypoints_gt.view(-1, 2).unsqueeze(0)  # Add an extra dimension to match expected shape for loss_edges

#     return keypoints_gt, valid_vis_all, valid_invis_all

# def process_batch_keypoints(batch_keypoints):
#     # Assuming batch_keypoints is a batch of keypoints tensors
#     # Each tensor in the batch has shape [N, 3] where N is the number of keypoints
#     # and each keypoint is represented as [x, y, visibility]

#     # Concatenate all keypoints and visibilities from the batch
#     all_keypoints = torch.cat([kp for kp in batch_keypoints])
#     visibilities = all_keypoints[:, 2]  # Extract visibility flags

#     valid_vis_all = (visibilities == 1).long().cuda()
#     valid_invis_all = (visibilities == 0).long().cuda()

#     keypoints_gt = all_keypoints[:, :2].float().cuda()  # Discard visibility flags
#     keypoints_gt = keypoints_gt.view(-1, 2)  # Reshape for consistency

#     return keypoints_gt, valid_vis_all, valid_invis_all

def process_batch_keypoints(target_dicts):
    # This function now expects target_dicts, a list of dictionaries containing keypoints information
    batch_size = len(target_dicts)
#     print(batch_size)

    # Initialize lists to store keypoints and visibilities for each image in the batch
    keypoints_list = []
    visibilities_list = []

    for dict_ in target_dicts:
        # Each keypoints tensor in the dict is expected to have a shape [num_keypoints, 3]
        keypoints = dict_['keypoints'].squeeze(1).to(device)
        print(f"Original shape of keypoints in dict: {keypoints.shape}")

        # Extract x, y coordinates and visibility flags
        xy_coords = keypoints[:, :2]  # Keep only x, y coordinates
        visibilities = keypoints[:, 2]  # Extract visibility flags

        keypoints_list.append(xy_coords)
        visibilities_list.append(visibilities)

    # Concatenate keypoints and visibilities for the entire batch
    # The final shape of keypoints_gt should be [batch_size, num_keypoints, 2]
    keypoints_gt = torch.stack(keypoints_list).float().cuda()
    visibilities = torch.stack(visibilities_list).cuda()

    # Create valid visibility masks
    valid_vis_all = (visibilities == 1).long().cuda()
    valid_invis_all = (visibilities == 0).long().cuda()
    return keypoints_gt, valid_vis_all, valid_invis_all

def reorder_batch_keypoints(batch_keypoints):
    # Assuming batch_keypoints is a tensor of shape [batch_size, num_keypoints, num_features]
    batch_size, num_keypoints, num_features = batch_keypoints.shape
    reordered_keypoints_batch = []

    for i in range(batch_size):
        # Directly use the normalized keypoints
        normalized_keypoints = batch_keypoints[i]

        # Initialize a tensor for reordered keypoints with only x, y coordinates
        reordered_normalized_keypoints = torch.zeros(num_keypoints, 2, device=batch_keypoints.device)

        # Reordering logic
        rounded_labels = torch.round(normalized_keypoints[:, -1]).int()
        used_indices = []
        for label in range(1, 7):
            valid_idx = (rounded_labels == label).nonzero(as_tuple=True)[0]
            if valid_idx.numel() > 0:
                reordered_normalized_keypoints[label - 1] = normalized_keypoints[valid_idx[0], :2]
            else:
                invalid_idx = ((rounded_labels < 1) | (rounded_labels > 6)).nonzero(as_tuple=True)[0]
                invalid_idx = [idx for idx in invalid_idx if idx not in used_indices]
                if invalid_idx:
                    reordered_normalized_keypoints[label - 1] = normalized_keypoints[invalid_idx[0], :2]
                    used_indices.append(invalid_idx[0])

        reordered_keypoints_batch.append(reordered_normalized_keypoints)

    return torch.stack(reordered_keypoints_batch)

def denormalize_keypoints(batch_keypoints, width=640, height=480):
    # Assuming batch_keypoints is a batch of normalized keypoints tensors
    # Denormalize each keypoint in the batch
    denormalized_keypoints = []
    for kp in batch_keypoints:
        denormalized_x = (kp[:, 0] * (width / 2)) + (width / 2)
        denormalized_y = (kp[:, 1] * (height / 2)) + (height / 2)
        denormalized_kp = torch.stack((denormalized_x, denormalized_y), dim=1)
        denormalized_keypoints.append(denormalized_kp)
        
    denormalized_keypoints = torch.stack(denormalized_keypoints)
#     print("denormalized_keypoints.shape", denormalized_keypoints.shape)
    return denormalized_keypoints

In [27]:
# Define the model
model = KeypointPipeline(weights_path)
model = model.to(device)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1 # Define your number of epochs
batch_size = 2

split_folder_path = train_test_split(root_dir)
KEYPOINTS_FOLDER_TRAIN = split_folder_path +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = split_folder_path +"/val"
KEYPOINTS_FOLDER_TEST = split_folder_path +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=None, demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

v = 1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for imgs, target_dicts, _ in data_loader_train:
        imgs = [img.to(device) for img in imgs]
        optimizer.zero_grad()

        # Forward pass for batch
        logits, KGNN2D, batch_labeled_keypoints = model(imgs)
#         print(KGNN2D.shape)
        print("Normalized Prediction in training", KGNN2D)

        # Process keypoints for the entire batch
        keypoints_gt, valid_vis_all, valid_invis_all = process_batch_keypoints(target_dicts)
        print("gt keypoints for loss", keypoints_gt)
#         print("valid_invis_all.shape", valid_invis_all.shape)
        
        # Normalize and reorder keypoints as per your existing logic
        # Ensure this logic works on the batch level
        
        reordered_normalized_keypoints = reorder_batch_keypoints(KGNN2D)
#         print("keypoints_logits.shape", reordered_normalized_keypoints.shape)
        # Denormalize the reordered keypoints for the entire batch
        print("Normalized Reordered", reordered_normalized_keypoints)
        denormalized_keypoints = denormalize_keypoints(reordered_normalized_keypoints)
        print("Denormalized Prediction in training", denormalized_keypoints)
        
#         print(valid_vis_all.shape)
#         print(logits.shape)
        
#         loss_kgnn2d = kgnn2d_loss(keypoints_gt, valid_invis_all, denormalized_keypoints)
        loss_kgnn2d = kgnn2d_loss(keypoints_gt, denormalized_keypoints)

        # Compute batch losses
        edge_loss = loss_edges(valid_vis_all, logits)
        
#         loss_kgnn2d = kgnn2d_loss(keypoints_gt, denormalized_keypoints, valid_vis_all)

        # Combine the losses
        total_batch_loss = edge_loss + loss_kgnn2d
        total_batch_loss.backward()
        optimizer.step()
        total_loss += total_batch_loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader_train)}')

Using factor graph MLP encoder.
Using learned graph decoder.


KeyboardInterrupt: 

In [ ]:
import torch
from torchvision.transforms import functional as F
from PIL import Image
import cv2
import numpy as np

# Assuming KeypointPipeline and all necessary classes/functions are defined above or imported

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Path to the trained model weights
model_path = '/home/jc-merlab/Pictures/Data/trained_models/occ_ckpt/ckpt_e100.pth'

# Load the model
model = torch.load(model_path)
model = model.to(device)
model.eval()  # Set the model to evaluation mode

def prepare_image(image_path):
    """
    Load an image and prepare it for the model prediction.
    This function should replicate the preprocessing applied during training.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Your preprocessing steps here, for example, resizing, normalization, etc.
    # This is a placeholder transformation, adjust it according to your training transformations
    img_tensor = F.to_tensor(img).to(device)  # Add batch dim and convert to tensor
    return img_tensor

def predict(model, img_tensor):
    # Assuming the model takes a list of tensors as input
    with torch.no_grad():
        logits, KGNN2D, batch_labeled_keypoints = model([img_tensor])
    # Process the output as needed
    return KGNN2D

def postprocess_keypoints(keypoints, width=640, height=480):
    # Adjust this function based on your model's output format
    denormalized_keypoints = denormalize_keypoints(keypoints, width, height)
    return denormalized_keypoints

image_path = '/home/jc-merlab/Pictures/Data/occ_sim_dataset/004866.rgb.jpg'


# Example usage
img_tensor = prepare_image(image_path).to(device)
KGNN2D = predict(model, img_tensor)
ordered_keypoints = reorder_batch_keypoints(KGNN2D)
denormalized_keypoints = postprocess_keypoints(ordered_keypoints)
print(denormalized_keypoints)

In [ ]:
"keypoints": [[[257.95220042652915, 366.9198630617724, 1]], [[257.95973939799904, 283.013113744617, 1]], 
              [[179.70016595863137, 298.244571585954, 1]], [[175.69899307811323, 277.8348649543144, 0]], 
              [[79.90486225732596, 303.90392338594796, 0]], 
              [[66.15504343164937, 289.4557892368882, 1]]]

In [14]:
import cv2
import os
import torch
from torchvision.transforms import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model
model_path = '/home/jc-merlab/Pictures/Data/trained_models/occ_ckpt_2/ckpt_e120.pth'
model = torch.load(model_path)
model = model.to(device)
model.eval()  # Set the model to evaluation mode

def prepare_image(image_path):
    """
    Load an image and prepare it for the model prediction.
    This function should replicate the preprocessing applied during training.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Your preprocessing steps here, for example, resizing, normalization, etc.
    # This is a placeholder transformation, adjust it according to your training transformations
    img_tensor = F.to_tensor(img).to(device)  # Add batch dim and convert to tensor
    return img_tensor


def predict(model, img_tensor):
    # Assuming the model takes a list of tensors as input
    with torch.no_grad():
        logits, KGNN2D, batch_labeled_keypoints = model([img_tensor])
    # Process the output as needed
    return KGNN2D

def postprocess_keypoints(keypoints, width=640, height=480):
    # Adjust this function based on your model's output format
    denormalized_keypoints = denormalize_keypoints(keypoints, width, height)
    return denormalized_keypoints

def visualize_keypoints(image_path, keypoints, out_dir):
    """
    Visualize keypoints on the image using cv2.
    """
    # Load the original image
    img = cv2.imread(image_path)
    # Convert keypoints to a NumPy array if it's a tensor
    if torch.is_tensor(keypoints):
        keypoints = keypoints.cpu().numpy()
    
    # Draw keypoints on the image
    for kp in keypoints[0]:  # Assuming the keypoints shape is [1, N, 2]
        x, y = int(kp[0]), int(kp[1])
        cv2.circle(img, (x, y), radius=9, color=(0, 0, 0), thickness=-1)
    
   # Construct the output path and save the image
    filename = os.path.basename(image_path)
    output_path = os.path.join(out_dir, filename)
    cv2.imwrite(output_path, img)

def process_folder(folder_path, output_path):
    """
    Process all images in the specified folder, predict and visualize keypoints.
    """
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            img_tensor = prepare_image(image_path).to(device)
            KGNN2D = predict(model, img_tensor)
            ordered_keypoints = reorder_batch_keypoints(KGNN2D)  # Ensure this function is defined
            denormalized_keypoints = postprocess_keypoints(ordered_keypoints)
            visualize_keypoints(image_path, denormalized_keypoints, output_path)

# Example usage
folder_path = '/home/jc-merlab/Pictures/Data/occ_test_data/'
output_path = '/home/jc-merlab/Pictures/Data/occ_test_data/output_attn/'
process_folder(folder_path, output_path)

Keypoints:  [[258, 367, 0.9999565, 1], [322, 308, 0.99733907, 5], [339, 296, 0.99838626, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [ 0.0063,  0.2833,  0.9973,  5.0000],
        [ 0.0594,  0.2333,  0.9984,  6.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.9999597, 1], [304, 214, 0.9984475, 4], [400, 197, 0.99874544, 5], [420, 186, 0.9967391, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.

Keypoints:  [[258, 367, 0.9999598, 1], [258, 283, 0.9999119, 2], [310, 223, 0.9992461, 3], [326, 237, 0.99979097, 4], [308, 332, 0.73011726, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.0312, -0.0708,  0.9992,  3.0000],
        [ 0.0188, -0.0125,  0.9998,  4.0000],
        [-0.0375,  0.3833,  0.7301,  6.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.999

Keypoints:  [[258, 367, 0.9999585, 1], [258, 283, 0.9999045, 2], [178, 277, 0.99977523, 3], [179, 256, 0.9997702, 4]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.4437,  0.1542,  0.9998,  3.0000],
        [-0.4406,  0.0667,  0.9998,  4.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.99995804, 1], [258, 283, 0.9999175, 2], [161, 121, 0.91463053, 3]]
graph fea

Keypoints:  [[258, 367, 0.9999583, 1], [258, 283, 0.9999424, 2], [399, 140, 0.7243855, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [ 0.2469, -0.4167,  0.7244,  6.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.9999579, 1], [258, 283, 0.99992466, 2], [305, 219, 0.999653, 3], [322, 231, 0.99981767, 4]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.00

Keypoints:  [[258, 367, 0.9999583, 1], [258, 283, 0.99989927, 2]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.99995923, 1], [178, 276, 0.9998584, 3], [180, 256, 0.9996958, 4], [157, 159, 0.9985258, 5], [179, 155, 0.9949804, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.4437,  0.1500,  0.9999,  3.0

...

Keypoints:  [[258, 283, 0.99991786, 2], [193, 231, 0.99745053, 3], [257, 377, 0.878238, 4], [304, 231, 0.9978387, 5]]
graph feature:  tensor([[-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.3969, -0.0375,  0.9975,  3.0000],
        [-0.1969,  0.5708,  0.8782,  4.0000],
        [-0.0500, -0.0375,  0.9978,  5.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.99995816, 1], [193, 237, 0.9864833, 3], [297, 177, 0.99881077, 5], [319, 174

Keypoints:  [[258, 367, 0.9999583, 1], [258, 283, 0.99990964, 2], [311, 223, 0.9994204, 3], [326, 238, 0.99977976, 4]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.0281, -0.0708,  0.9994,  3.0000],
        [ 0.0188, -0.0083,  0.9998,  4.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.9999583, 1], [258, 283, 0.9999386, 2], [301, 220, 0.77162695, 3]]
graph fea

shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.99995816, 1], [258, 283, 0.9999385, 2], [305, 149, 0.9609406, 6]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  0.9999,  2.0000],
        [-0.0469, -0.3792,  0.9609,  6.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.999958

Keypoints:  [[258, 367, 0.99995816, 1], [258, 283, 0.9999583, 2], [211, 218, 0.9998518, 3], [228, 206, 0.9996729, 4]]
graph feature:  tensor([[-0.1938,  0.5292,  1.0000,  1.0000],
        [-0.1938,  0.1792,  1.0000,  2.0000],
        [-0.3406, -0.0917,  0.9999,  3.0000],
        [-0.2875, -0.1417,  0.9997,  4.0000]], device='cuda:0')
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
shape of x before attention torch.Size([1, 6, 512])
shape of x after attn torch.Size([1, 6, 512])
batch norm input torch.Size([1, 6, 512])
shape of x before attention torch.Size([1, 10, 512])
shape of x after attn torch.Size([1, 10, 512])
batch norm input torch.Size([1, 10, 512])
Keypoints:  [[258, 367, 0.9999584, 1], [258, 283, 0.9999597, 2], [211, 218, 0.99988246, 3], [228, 206,

KeyboardInterrupt: 